Scaffolding of Tensorflow
===

```
Teosorflow version: 1.3.0
Python Version: 3.6.1
```
since tenesorflow evolves vert fast, check above before using:

```python
import tensorflow as tf

# initialize variables/model parameters, 模型初始化

# define the training loop operations, 訓練步驟
def inference(X):
    # compute inference model over data X and return the result
    # 輸入值 X 的推論
    return

def loss(X, Y):
    # compute loss over training data X and expected values Y
    # 誤差函數
    return

def inputs():
    # read/generate input training data X and expected outputs Y
    # 訓練數據 X
    return

def train(total_loss):
    # train / adjust model parameters according to computed total loss
    # 模型訓練運算
    return

def evaluate(sess, X, Y):
    # evaluate the resulting trained model
    # 結果估計
    return


# Launch the graph in a session, setup boilerplate, Tensorflow 運算圖
with tf.Session() as sess:

    tf.global_variables_initializer().run()

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 1000
    steps_no=10
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % steps_no == 0:
            print("At",step+steps_no,"step, loss: ", sess.run([total_loss]))

    evaluate(sess, X, Y)

    coord.request_stop()
    coord.join(threads)
    sess.close()
```



Titanic Example
---

In [ ]:
import tensorflow as tf
import os,time

print(tf.__version__)

**1.** Inputs Combining

**a).** $X\cdot W+b$

In [ ]:
# same params and variables initialization as log reg.
W = tf.Variable(tf.zeros([3, 1]), name="weights")
b = tf.Variable(0., name="bias")

# former inference is now used for combining inputs
def combine_inputs(X):
    return tf.matmul(X, W) + b

**b).** Inference by applying sigmod function 
$$\mathbf{y_{\text{prediction}}=f(X)=\frac{1}{1+e^{-(X\cdot W+b)}}}$$

In [ ]:
# new inferred value is the sigmoid applied to the former
def inference(X):
    return tf.sigmoid(combine_inputs(X))

**2.** Loss function
$$\mathbf{Loss =\sum(Y_i-Y_{i,pred})^2}$$ 

In [ ]:
def loss(X, Y):
    Y_pred = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y,Y_pred))


**3.** Data Handling

**a)**. Load data


In [ ]:
def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)

**b).** Determine what features and labels are used:

In [ ]:
def inputs():
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        read_csv(100, "data/Titanic/train.csv",  \
        [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])

    gender = tf.to_float(tf.equal(sex, ["female"]))
    pclass = tf.to_float(pclass)

    # Finally we pack all the features in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([pclass, gender, age]))
    survived = tf.reshape(survived, [100, 1])

    return features, survived

**4.** Training Method

In [ ]:
def train(total_loss,learning_rate=0.01):
    #learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

**5.** Model Evaulation 

In [ ]:
def evaluate(sess, X, Y):

    predicted = tf.cast(inference(X) > 0.5, tf.float32)

    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

**6.** Run the tensorflow Model

In [ ]:
startTime = time.time()
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    #init=tf.global_variables_initializer()
    #sess.run(init)
    tf.global_variables_initializer().run()
   
    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 1000
    steps_no = 100
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % steps_no == 0:
            print("At",step+steps_no, "step, loss: ", sess.run([total_loss]),)

    evaluate(sess, X, Y)

    #import time
    #time.sleep(5)

    coord.request_stop()
    coord.join(threads)
    sess.close()
    print("Time taken: %f" % (time.time() - startTime))

# Performace Improvements
1. Well conversation of Data
-  Adjection method of Loss function and parameters